[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Image%20Statistics/ImageStatisticsTutorial_forClass2025_part2.ipynb)

In [ ]:
# Image statistics tutorial; part 2
# This is a tutorial and does not include questions at the end.
# Image statistics through filters and Gaussian Scale Mixture model
# OS 2025. Original in Matlab by OS used in Berkeley summer course.

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic_2d
from scipy.stats import rayleigh
import sys
import os


In [ ]:
!wget https://github.com/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Convolutional%20Neural%20Network/ILSVRC2012_test_00026783.JPEG?raw=true -O ILSVRC2012_test_00026783.JPEG
# load an image
from tensorflow.keras.preprocessing import image
import numpy as np
img_path = 'ILSVRC2012_test_00026783.JPEG'
img = image.load_img(img_path, target_size=(224, 224))
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
# Create a Gabor filter and look at the statistics of an image
# through the Gabor filter

def mkSine(fsz, period, direction, amplitude, phase, shift):
    """
    Create a sine wave of size `fsz`, period `period`, oriented along `direction`,
    scaled by `amplitude`, with phase offset `phase` and shift `shift`.
    """
    y, x = np.meshgrid(np.arange(fsz), np.arange(fsz))
    x_shifted = x - shift[0]
    y_shifted = y - shift[1]
    x_rot = x_shifted * np.cos(direction) + y_shifted * np.sin(direction)
    sine_wave = amplitude * np.sin(2 * np.pi * x_rot / period + phase)
    return sine_wave

def mkGaussian(fsz, sigma, center, amplitude):
    """
    Create a Gaussian filter of size `fsz`, standard deviation `sigma`,
    centered at `center`, scaled by `amplitude`.
    """
    y, x = np.meshgrid(np.arange(fsz), np.arange(fsz))
    gauss = amplitude * np.exp(-((x - center[0])**2 + (y - center[1])**2) / (2 * sigma**2))
    return gauss

# Make a Gabor filter; run multiple times to see different Gabors
# or change the parameters

fsz = 15;
ctr = [round(fsz / 2), round(fsz / 2)]
the_period = 4
the_sig = max(2, int(np.ceil(np.random.rand() * the_period)))
the_dir = np.pi/5
direction = np.sign(0.5 - the_dir) * np.random.rand() * 2 * np.pi
phase = np.random.rand() * 2 * np.pi
the_sine = mkSine(fsz, the_period, direction, 1, phase, (fsz + 1) * np.random.rand(2))
plt.subplot(2,2,1)
plt.imshow(the_sine, cmap='gray')
plt.title('Sinusoid')

the_gauss = mkGaussian(fsz, the_sig, ctr, 1)
plt.subplot(2,2,2)
plt.imshow(the_gauss, cmap='gray')
plt.title('Gaussian')

the_filt = the_sine * the_gauss
plt.subplot(2,2,3)
plt.imshow(the_filt, cmap='gray')
plt.title('Gabor')


In [ ]:
# auxiliary functions for image statistics; we will use these later in the tutorial

# Functions for examining image statistics
# plotting joint conditional dependencies (bowties)
# Schwartz and Simoncelli 2001
# The original paper used steerable wavelet filters. Here we are using Gabor filters or
# generating synthetic stimuli.

def range2(x):
    return np.min(x), np.max(x)

def mean2(x):
    return np.mean(x)

# joint histogram
def jhisto(Yim, Xim, nbins=None, binctr=None):
  # based on eps Matlab function
    if nbins is None:
        nbins = np.array([101, 101])

    nbins = np.abs(np.round(nbins))
    if np.prod(np.shape(nbins)) == 1:
        nbins = np.array([nbins, nbins])
    else:
        nbins = np.reshape(nbins, (-1,))

    if binctr is None:
        binctr = [mean2(Yim), mean2(Xim)]

    nbins = nbins - 1
    Xmn, Xmx = range2(Xim)
    X = np.linspace(Xmn, Xmx, nbins[1] + 1)
    Ymn, Ymx = range2(Yim)
    Y = np.linspace(Ymn, Ymx, nbins[0] + 1)
    Ybs = (Y[1] - Y[0]) / 2
    nbins = nbins + 1
    N = np.zeros(nbins)
    for yind in range(nbins[0]):
        ind1 = np.where(Yim > (Y[yind] - Ybs))
        ind2 = np.where(Yim[ind1] < (Y[yind] + Ybs))
        ind = ind1[0][ind2]
        if ind.size > 0:
            N[yind, :], _ = np.histogram(Xim[ind], bins=nbins[1], range=(Xmn, Xmx))

    return N, Y, X

# for computing joint conditional statistics
def bowtie(n1, n2, binsz=51):
    H, Y, X = jhisto(n1, n2, binsz)
    colmax = np.maximum(1, np.max(H, axis=0))
    H = H / colmax
    axis_min = X[0]*.7
    axis_max = X[-1]*.7
    plt.imshow(H, aspect='auto', origin='lower', extent=[axis_min, axis_max, axis_min, axis_max], cmap='gray')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.xlim(axis_min, axis_max)
    plt.ylim(axis_max, axis_max)
    plt.axis('square')
    plt.axis('equal')
    plt.show()

In [ ]:
# Convolution of the image with the Gabor filter

from scipy.signal import convolve2d

img = image.load_img(img_path, target_size=(224, 224));
x = image.img_to_array(img);
x = x[:,:,0];
print(x.shape)
convolved_output = convolve2d(x, the_filt, mode='same', boundary='fill', fillvalue=0)
plt.imshow(convolved_output, cmap='gray')
plt.title('convolved output')


In [ ]:
# histogram of the image convolved with the filter
# This is the statistics of the image through a single filter (i.e., the marginal statistics)

plt.hist(convolved_output.flatten(), bins=64, density=True)
plt.title('histogram')

In [ ]:
# Joint conditional statistics between two filter responses to images
# Make two Gabor filters
# and look at statistics of image through the two filters
# You can also change the direction (the_dir) or other parameters to
# see the effects

# Make a Gabor filter
fsz = 15
the_period = 4
the_dir = .2
direction = np.sign(0.5) * the_dir * 2 * np.pi
phase = 0;
the_sine1 = mkSine(fsz, the_period, direction, 1, phase, (fsz + 1)*[0,0])

the_gauss = mkGaussian(fsz, the_sig, ctr, 1)
the_filt1 = the_sine1 * the_gauss
plt.subplot(2,3,1)
plt.imshow(the_filt1, cmap='gray')
plt.title('Gabor 1')

# Make another Gabor filter
the_dir = .15
direction = np.sign(0.5) * the_dir * 2 * np.pi
phase = np.pi/4;
the_sine2 = mkSine(fsz, the_period, direction, 1, phase, (fsz + 1)*[0,0])

the_filt2 = the_sine2 * the_gauss
plt.subplot(2,3,2)
plt.imshow(the_filt2, cmap='gray')
plt.title('Gabor 2')

convolved_output1 = convolve2d(x, the_filt1, mode='same', boundary='fill', fillvalue=0)
convolved_output2 = convolve2d(x, the_filt2, mode='same', boundary='fill', fillvalue=0)
plt.subplot(2,3,3)
plt.imshow(convolved_output1, cmap='gray')
plt.title('Gabor 1 response')
plt.subplot(2,3,4)
plt.imshow(convolved_output2, cmap='gray')
plt.title('Gabor 2 response')

# We use the bowtie auxiliary function to generate a two dimensional joint conditional histogram between
# the outputs of filter 1 and filter 2. We call this a "bowtie" because of its shape.
# Pixel intensity is proportional to the bin counts, except that each column is independently rescaled
# to fill the range of intensities. The variance of distribution of the filter 2 response increases with
# increasing value (both positive and negative) of the responses of filter 1.
plt.subplot(2,3,6)
plt.title('Bowtie statistics')
bowtie(convolved_output.flatten(), convolved_output2.flatten(), 64)

In [ ]:
# We can generate two vectors that have the same type of statistics we observe when we look
# at the responses of images through filters. We will use a Gaussian Scale Mixture (GSM) model to do this.
# GSM papers background, see: Cagli, Schwartz,Kohn 2015; Schwartz, Sejnowski, Dayan 2009 (also Wainwright and Simoncelli 1999, Andrews and Mallows)

# Synthetic generated example according to Gaussian Scale Mixture Model (GSM)
# Multiplying two independent Gaussians g1 and g2 by a common (here Rayleigh distributed) mixer variable
# to create a statistical variance (multiplicative) dependency, and sparse marginals

# Note: One could also make the Gaussians linearly correlated for a tilted bowtie, which we will not to here

numsamples_synth = 1000000
# Generate two independent Gaussian variables
g1 = np.random.randn(numsamples_synth)
g2 = np.random.randn(numsamples_synth)
scale = .1

# Generate random variables based on the Rayleigh distribution
mixer = rayleigh.rvs(scale, size=numsamples_synth);
print(mixer.shape);
bowtie(g1,g2)

# We multiply each independent Gaussian by the same mixer variable, to introduce
# a multiplicative (bowtie) dependency
l1 = g1*mixer;
l2 = g2*mixer;
bowtie(l1,l2)
plt.hist(l1, bins=64, density=True)
print(np.dot(np.transpose(g1),g2)/numsamples_synth)
print(np.dot(np.transpose(abs(l1)),abs(l2))/numsamples_synth)


In [ ]:
# We have thus far looked at joint conditional histograms (bowties).
# Another way to observe the joint statistics is to look at joint contour histograms

# contour plots for the Gaussians
# joint histogram
N, Y, X = jhisto(g1, g2, 32);
# Normalize the distribution to sum to 1
print(np.sum(np.sum(N)))
N = N / np.sum(np.sum(N))
print(np.sum(np.sum(N)))
print(X.shape)
print(Y.shape)
print(N.shape)

# Plot the contour
plt.subplot(2, 2, 1)
theval = np.max(N)
the_epsilon = 10 ** -5
contour = plt.contour(X, Y, np.log2(N + the_epsilon), levels=8)
plt.axis('square')




In [ ]:
# Here we will again repeat plotting the joint contour plots for the two Gaussians
# (which is the joint distribution of the two Gaussian vectors we made, plotted
# as contours). We will compare this to a product of the marginal distribution, i.e.
# the distribution of the first Gaussian times the distribution of
# the second Gaussian. If you remember your probability, for two variables that
# are independent, the joint distribution equals the product of the marginals
# P(g1,g2) = P(g1)P(g2)

# contour plots for the output of the Gaussians
# joint histogram
N, Y, X = jhisto(g1, g2, 32);
# Normalize the distribution to sum to 1
print(np.sum(np.sum(N)))
N = N / np.sum(np.sum(N))
print(np.sum(np.sum(N)))
print(X.shape)
print(Y.shape)
print(N.shape)

# Plot the contour
plt.figure();
plt.subplot(2, 2, 1)
plt.title('joint')
theval = np.max(N)
the_epsilon = 10 ** -5
contour = plt.contour(X, Y, np.log2(N + the_epsilon), levels=8)
plt.axis('square')
plt.axis([-5,5,-5,5])

# Compare the joint histograms to the product of the marginals
# for the Gaussians (should be the same! -- why?)
plt.subplot(2, 2, 3)
h1, bins1, _ = plt.hist(g1, 32);
h2, bins2, _ = plt.hist(g2, bins1);     # we want to use the same bins for both distributions

# normalize the distributions to sum to 1
h1 = h1 / np.sum(h1)
h2 = h2 / np.sum(h2)
print(h1.shape)

bin_centers1 = (bins1[:-1] + bins1[1:]) / 2
bin_centers2 = (bins2[:-1] + bins2[1:]) / 2
print(h1)
print(bins1)
print(bin_centers1)
print(bins1.shape)
print(bin_centers1.shape)

# product of marginals
prod_marginals = np.outer(h1, h2)
prod_marginals = prod_marginals / np.sum(np.sum(prod_marginals))
print(prod_marginals.shape)

plt.subplot(2, 2, 2);
plt.title('product of marginals')
contour = plt.contour(bin_centers1, bin_centers2, np.log2(prod_marginals + the_epsilon), levels=8)
plt.axis('square')



In [ ]:
# contour plots for the output of the Gaussian Scale Mixture
# joint histogram
N, Y, X = jhisto(l1, l2, 32);
# Normalize the distribution to sum to 1
print(np.sum(np.sum(N)))
N = N / np.sum(np.sum(N))
print(np.sum(np.sum(N)))
print(X.shape)
print(Y.shape)
print(N.shape)

# Plot the contour
plt.figure();
plt.subplot(2, 2, 1)
plt.title('joint')
theval = np.max(N)
the_epsilon = 10 ** -5
contour = plt.contour(X, Y, np.log2(N + the_epsilon), levels=8)
plt.axis('square')

# Compare the joint histograms to the product of the marginals
# for the Gaussian Scale Mixture (why do they have a different shape?)
plt.subplot(2, 2, 3)
h1, bins1, _ = plt.hist(l1, 32);
h2, bins2, _ = plt.hist(l2, bins1);

# normalize the distributions to sum to 1
h1 = h1 / np.sum(h1)
h2 = h2 / np.sum(h2)
print(h1.shape)

bin_centers1 = (bins1[:-1] + bins1[1:]) / 2
bin_centers2 = (bins2[:-1] + bins2[1:]) / 2
print(h1)
print(bins1)
print(bin_centers1)
print(bins1.shape)
print(bin_centers1.shape)

# product of marginals
prod_marginals = np.outer(h1, h2)
prod_marginals = prod_marginals / np.sum(np.sum(prod_marginals))
print(prod_marginals.shape)

plt.subplot(2, 2, 2);
plt.title('product of marginals')
contour = plt.contour(bin_centers1, bin_centers2, np.log2(prod_marginals + the_epsilon), levels=8)
plt.axis('square')

